In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt
from google.colab import drive
import cv2 as cv
from datetime import datetime
import math
from PIL import Image
from sklearn.preprocessing import LabelEncoder


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:


train_dir = "drive/MyDrive/CS 539/Term Project/fruits-360_dataset/fruits-360/Training"
test_dir = "drive/MyDrive/CS 539/Term Project/fruits-360_dataset/fruits-360/Test"
project_dir = "drive/MyDrive/CS 539/Term Project"
checkpoint_dir = "drive/MyDrive/CS 539/Term Project/Checkpoints"

def join_path(list):
  return "/".join(list)

In [7]:
# Get the list of labels
labels = [name for name in os.listdir(train_dir)]


In [ ]:
# Create my train/test X/Y sets
images_x  =  []       
labels_y  =  [] 

for label in os.listdir(train_dir):
  for filename in os.listdir(join_path([train_dir, label])):
    # print(label, filename)
    # print(join_path([train_dir, label, filename]))
    img  =  cv.imread(join_path([train_dir, label, filename]))
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img_arr = Image.fromarray(img)
    images_x.append(np.array(img_arr))
    labels_y.append(label)


In [ ]:
print(len(images_x))
print(len(labels_y))

In [ ]:
# Encode the labels
lb_encod  =  LabelEncoder()
labels_y = pd.DataFrame(labels_y)
labels_y = lb_encod.fit_transform(labels_y[0])

In [ ]:
print(len(labels_y))
print(labels_y)

In [ ]:

# Check an image
figure = plt.figure(figsize = (8,8))
ax = figure.add_subplot(121)
ax.imshow(images_x[0])
bx = figure.add_subplot(122)
bx.imshow(images_x[60])
plt.show()


In [ ]:
#Saving the image array and corresponding labels
images_x = np.array(images_x)
np.save(join_path([checkpoint_dir, "images"]),images_x)
np.save(join_path([checkpoint_dir, "labels"]),labels_y)



In [7]:
#Loading the images and labels that we have saved above
images_x = np.load(join_path([checkpoint_dir, "images.npy"]),allow_pickle = True)
labels_y = np.load(join_path([checkpoint_dir, "labels.npy"]),allow_pickle = True)

# Shuffle the data
img_shape  = np.arange(images_x.shape[0])
np.random.shuffle(img_shape)
image = images_x[img_shape]
labels_y = labels_y[img_shape]


In [8]:
# Split the train and test data sets
num_classes = len(np.unique(labels_y))
len_data = len(images_x)
x_train, x_test = images_x[(int)(0.1*len_data):],images_x[:(int)(0.1*len_data)]
y_train,y_test = labels_y[(int)(0.1*len_data):],labels_y[:(int)(0.1*len_data)]

import keras
y_train = keras.utils.to_categorical(y_train,num_classes)
y_test = keras.utils.to_categorical(y_test,num_classes)


In [9]:
print(x_train.shape)
print(y_train.shape)

(60923, 100, 100, 3)
(60923, 131)


In [ ]:
# VGG16

In [24]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.layers import Activation, Convolution2D, Dropout, Conv2D,AveragePooling2D, BatchNormalization,Flatten,GlobalAveragePooling2D
from keras import layers
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping



In [11]:
base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(100,100, 3)
    )

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# image = load_img(join_path([train_dir, "Apple Braeburn", "0_100.jpg"]))
# image = img_to_array(image)
# image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
# image = preprocess_input(image)
# yhat = model.predict(image) 
# label = decode_predictions(yhat)
# label = label[0][0]
# print('%s (%.2f%%)' % (label[1], label[2]*100)) 



In [21]:
base_model.trainable=False
for layer in base_model.layers:
  layer.trainable = False

In [22]:
last_layer = base_model.get_layer("block5_pool")
last_output = last_layer.output
x = Flatten()(last_output)
x = Dense(512, activation='relu', name='FC_2')(x) 
x = BatchNormalization()(x) 
x = Dropout(0.5)(x) 
x = Dense(131, activation='softmax', name='softmax')(x) 
new_model = Model(inputs=base_model.input, outputs=x) 
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0   

In [ ]:
new_model.compile(
    Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
     metrics=['accuracy']
     )
# new_model.fit_generator(train_batches, steps_per_epoch=4,
#  validation_data=valid_batches, validation_steps=2,
#  epochs=20, verbose=2)
history = new_model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=5,
    verbose=1,
    validation_split = 0.33,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)]
    )


Epoch 1/5
319/319 [==============================] - 5880s 18s/step - loss: 6.0240 - accuracy: 0.0081 - val_loss: 5.3287 - val_accuracy: 0.0072
Epoch 2/5
319/319 [==============================] - 5542s 17s/step - loss: 5.6235 - accuracy: 0.0099 - val_loss: 5.1828 - val_accuracy: 0.0080
Epoch 3/5
319/319 [==============================] - 5518s 17s/step - loss: 5.4563 - accuracy: 0.0118 - val_loss: 5.1123 - val_accuracy: 0.0074
Epoch 4/5
319/319 [==============================] - 5637s 18s/step - loss: 5.3212 - accuracy: 0.0151 - val_loss: 5.1033 - val_accuracy: 0.0095
Epoch 5/5
145/319 [============>.................] - ETA: 35:57 - loss: 5.2072 - accuracy: 0.0172

In [ ]:
# InceptionV3

In [67]:

# read in libraries
import tensorflow as tf
from tensorflow.keras import backend, models, layers, optimizers
import numpy as np
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from IPython.display import display
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os, shutil
from tensorflow.keras.models import Model
# Load InceptionV3 library
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Always clear the backend before training a model
backend.clear_session()
# InceptionV3 model and use the weights from imagenet
conv_base = InceptionV3(weights = 'imagenet', #Useing the inception_v3 CNN that was trained on ImageNet data.  
                  include_top = False)
conv_base.trainable=False

In [68]:
# Connect the InceptionV3 output to the fully connected layers
InceptionV3_model = conv_base.output
pool = GlobalAveragePooling2D()(InceptionV3_model)
dense_1 = layers.Dense(1024, activation = 'relu')(pool)
dense_2 = layers.Dense(512, activation = 'relu')(dense_1)
output = layers.Dense(131, activation = 'softmax')(dense_2)


In [ ]:
# Create an example of the Archictecture to plot on a graph
model_example = models.Model(inputs=conv_base.input, outputs=output)
# plot graph
plot_model(model_example)

In [69]:
# Define/Create the model for training
model_InceptionV3 = models.Model(inputs=conv_base.input, outputs=output)
# Compile the model with categorical crossentropy for the loss function and SGD for the optimizer with the learning
# rate at 1e-4 and momentum at 0.9
model_InceptionV3.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy'],
    )


In [65]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 172993822524904783
xla_global_id: -1
]


In [ ]:
# Execute the model with fit_generator within the while loop utilizing the discovered GPU
import tensorflow as tf
with tf.device("/device:CPU:0"):
  history = model_InceptionV3.fit(
      x_train,
      y_train,
      batch_size=128,
      epochs=5,
      verbose=1,
      validation_split = 0.33,
      callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)]
      )

    # history = model_InceptionV3.fit_generator(
    #     train_generator,
    #     epochs=5,
    #     validation_data=test_generator,
    #     verbose = 1,
    #     callbacks=[EarlyStopping(monitor='val_accuracy', patience = 5, restore_best_weights = True)])

In [ ]:
# Custom hand-crafted from scratch

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Dropout,Flatten,MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.layers import Activation, Convolution2D, Dropout, Conv2D,AveragePooling2D, BatchNormalization,Flatten,GlobalAveragePooling2D
from keras import layers
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau

l2_reg = 0.001
opt = Adam(lr = 0.001)
s
#Defining the CNN Model
cnn_model  =  Sequential()
cnn_model.add(Conv2D(filters = 32, kernel_size = (2,2), input_shape = (100,100, 3), activation = 'relu',kernel_regularizer = l2(l2_reg)))
cnn_model.add(MaxPool2D(pool_size = (2,2)))
cnn_model.add(Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu',kernel_regularizer = l2(l2_reg)))
cnn_model.add(MaxPool2D(pool_size = (2,2)))
cnn_model.add(Conv2D(filters = 128, kernel_size = (2,2), activation = 'relu',kernel_regularizer = l2(l2_reg)))
cnn_model.add(MaxPool2D(pool_size = (2,2)))
cnn_model.add(Dropout(0.1))

cnn_model.add(Flatten())

cnn_model.add(Dense(1048, activation = 'relu'))
cnn_model.add(Dense(262, activation = 'relu'))
cnn_model.add(Dense(131, activation = 'softmax'))

#CNN Model Summary
cnn_model.summary()


In [ ]:
#Compiling the model
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])


In [46]:
#Training the CNN Model
weights_file = "weights.hdf5"
checkpoint = ModelCheckpoint(join_path([checkpoint_dir, weights_file]), monitor='loss', verbose=1, save_best_only=True, mode='min')
history = cnn_model.fit(x_train,y_train,batch_size = 128,epochs = 110,verbose = 1,validation_split = 0.33)


Epoch 1/110
 26/319 [=>............................] - ETA: 11:19 - loss: 67.8938 - accuracy: 0.0045

KeyboardInterrupt: ignored

In [ ]:
#Check the performance
scores  =  cnn_model.evaluate(x_test, y_test, verbose = 1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])


In [ ]:
#Visualize the performance
figure = plt.figure(figsize = (10,5))
ax = figure.add_subplot(121)
ax.plot(history.history['accuracy'])
ax.plot(history.history['val_accuracy'])
ax.legend(['Training Accuracy','Val Accuracy'])
bx = figure.add_subplot(122)
bx.plot(history.history['loss'])
bx.plot(history.history['val_loss'])
bx.legend(['Training Loss','Val Loss'])

In [ ]:
#Test
test_path  =  'gdrive/My Drive/Dataset/Fruit Images/test_zip/test'
t_labels = []
t_images = []
for filename in os.listdir('gdrive/My Drive/Dataset/Fruit Images/test_zip/test'):
    if filename.split('.')[1]  =  =  'jpg':
        img  =  cv2.imread(os.path.join(test_path,filename))
        arr = Image.fromarray(img,'RGB')
        img_arr = arr.resize((50,50))
        t_labels.append(filename.split('_')[0])
        t_images.append(np.array(img_arr))

test_images = np.array(test_images)
np.save("test_image",test_images)
test_image = np.load("image.npy",allow_pickle = True)

pred = np.argmax(model.predict(test_image),axis = 1)
prediction  =  la.inverse_transform(pred)

test_image = np.expand_dims(test_image[25],axis = 0)
pred_test = np.argmax(model.predict(test_image),axis = 1)
prediction_test  =  la.inverse_transform(pred_test)

print(prediction_test[0])
plt.imshow(test_images[11])
